importing essential libraries

In [18]:
import requests
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense,Flatten,LSTM,Dropout
from keras.metrics import categorical_crossentropy
import timeit
from sklearn import metrics
from datetime import datetime

importing data direct from cloud

In [19]:
colors = '''aliceblue, antiquewhite, aqua, aquamarine, azure,
beige, bisque, black, blanchedalmond, blue,
blueviolet, brown, burlywood, cadetblue,
chartreuse, chocolate, coral, cornflowerblue,
cornsilk, crimson, cyan, darkblue, darkcyan,
darkgoldenrod, darkgray, darkgrey, darkgreen,
darkkhaki, darkmagenta, darkolivegreen, darkorange,
darkorchid, darkred, darksalmon, darkseagreen,
darkslateblue, darkslategray, darkslategrey,
darkturquoise, darkviolet, deeppink, deepskyblue,
dimgray, dimgrey, dodgerblue, firebrick,
floralwhite, forestgreen, fuchsia, gainsboro,
ghostwhite, gold, goldenrod, gray, grey, green,
greenyellow, honeydew, hotpink, indianred, indigo,
ivory, khaki, lavender, lavenderblush, lawngreen,
lemonchiffon, lime, limegreen,
linen, magenta, maroon, mediumaquamarine,
mediumblue, mediumorchid, mediumpurple,
mediumseagreen, mediumslateblue, mediumspringgreen,
mediumturquoise, mediumvioletred, midnightblue,
mintcream, mistyrose, moccasin, navajowhite, navy,
oldlace, olive, olivedrab, orange, orangered,
orchid, palegoldenrod, palegreen, paleturquoise,
palevioletred, papayawhip, peachpuff, peru, pink,
plum, powderblue, purple, red, rosybrown,
royalblue, saddlebrown, salmon, sandybrown,
seagreen, seashell, sienna, silver, skyblue,
slateblue, slategray, slategrey, snow, springgreen,
steelblue, tan, teal, thistle, tomato, turquoise,
violet, wheat, white, whitesmoke, yellow,
yellowgreen'''
colors = colors.replace(' ','')
colors = colors.replace('\n','')
colors = colors.split(',')

Importing data directly from cloud storage

In [22]:
r = requests.get('https://vor76f0raj.execute-api.ap-south-1.amazonaws.com/dev/listFiles')
files = r.json()
files.reverse()
counter = 0
for f in files:
    f['index'] = counter
    f['filePath'] = f['filePath'].replace(' ', '+')
    counter += 1

def pf(fm):
  points = str(fm['dataPoints']) if 'dataPoints' in fm else 'N/A'
  interval = str((fm['endTime'] - fm['startTime']) // fm['dataPoints']) if (points != 'N/A') else 'N/A'
  print(str(fm['index']) + '.\t' + points + '\t' + interval + '\t' + fm['user'].strip() + '\t' + fm['activity'] + '\t' + str(datetime.fromtimestamp(fm['startTime'] // 1000)))

def search(text):
  for f in files:
    if text in f['user'] or text in f['activity']:
      pf(f)


importing data of a particular index

In [4]:
def read_data(fileindex):
  data=pd.read_csv(files[fileindex]['filePath'])
  
  return data[["accelerox","acceleroy","acceleroz","gyrox","gyroy","gyroz"]]
  

trimming data from the beginning because at the beginning there are some 0 values and also shape of both running and walking training data becomes same by doing this

In [12]:
def filter_df(data):
  for col in ["accelerox","acceleroy","acceleroz","gyrox","gyroy","gyroz"]:

    data[col]=np.around(data[col]*100,0)
    data[col]=data[col]/100
  return data 
def trim(data,val):
 l1=int((data.shape[0]-val)/2)
 l=data.shape[0]-val-l1

 d1=list(np.arange(val+l,data.shape[0]))

 d2=list(np.arange(0,l))

 data.drop(d1,inplace=True)
 data.drop(d2,inplace=True)

 return data 

reshaping data into timestamps

In [6]:
def shaping(data,number): 
 l=[]
 for n in range(int(data.shape[0]/number)):
   l.append(data[number*n:number*(n+1)])
 hoho=np.array(l)
 return hoho

importing data

In [7]:
def im_data(tw,tr,te,vw,vr,number):
  
    
 
 trainwalk=filter_df(read_data(tw))
 trainrun=filter_df(read_data(tr))
 validwalk=filter_df(read_data(vw))
 validrun=filter_df(read_data(vr))    
 xtest=filter_df(read_data(te)) 
 

 val=20000

 walk=trim(trainwalk,val)
 run=trim(trainrun,val)
 

 

 val=5000

 vwalk=trim(validwalk,val)
 vrun=trim(validrun,val)
 

 val=5000
 xtest=trim(xtest,val)

 
 

 xvalid=np.array(pd.concat([vwalk,vrun]))
 xtrain=np.array(pd.concat([walk,run]))
 xtest=np.array(xtest)
   
 
 xvalid=shaping(xvalid,number)
 xtrain=shaping(xtrain,number) 
 xtest=shaping(xtest,number)



 return (xtrain,xtest,xvalid)

creating output labels

In [8]:
def label_create(trainsize,testsize,validsize,cho):
    ytrain=list(np.zeros((1,int(trainsize/2))).ravel().astype("int"))
    ytrain+=list(np.ones((1,int(trainsize/2))).ravel().astype("int"))
    ytrain=keras.utils.to_categorical(np.array(ytrain))
    

    yvalid=list(np.zeros((1,int(validsize/2))).ravel().astype("int"))
    yvalid+=list(np.ones((1,int(validsize/2))).ravel().astype("int"))
    yvalid=keras.utils.to_categorical(np.array(yvalid))    
    
    if cho==1:
     ytest=[0]
     ytest+=list(np.ones((1,testsize)).ravel().astype("int")) 
     ytest=keras.utils.to_categorical(np.array(ytest)) 
     ytest=np.delete(ytest,[0],axis=0)
    if cho==0:
     ytest=[1]
     ytest+=list(np.zeros((1,testsize)).ravel().astype("int")) 
     ytest=keras.utils.to_categorical(np.array(ytest)) 
     ytest=np.delete(ytest,[0],axis=0)    
     
    return (ytrain,ytest,yvalid)

creating model with 2 LSTM 1 Dense layer

In [9]:
def model_create(number):
    model=Sequential()
    model.add(LSTM(256,input_shape=(number,6),activation="relu",batch_size=1))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(2,activation="softmax"))
    model.compile(Adam(lr=0.00003),loss="categorical_crossentropy",metrics=["accuracy"])
    
    print(model.summary())
    return model


    




applying model

In [13]:
def model_fit(model,xtrain,xvalid,ytrain,yvalid):
    model.fit(xtrain,ytrain,epochs=10,verbose=2,batch_size=1,validation_data=(xvalid,yvalid))
    return model

In [14]:
def model_test(model,xtrain,xtest,ytrain,ytest):
    predictions = model.predict(xtest,batch_size=1,verbose=0)  
    return (ytest,predictions,metrics.accuracy_score(ytest.argmax(axis=1), predictions.argmax(axis=1))) 

Driver function

In [15]:
def allwork():
   tw=283
   tr=282
   vw=269
   vr=268
   te=269
   tt="w"
   dic={"w":0,"r":1}
   number=100

   
   xtrain,xtest,xvalid=im_data(tw,tr,te,vw,vr,number)
   ytrain,ytest,yvalid=label_create(xtrain.shape[0],xtest.shape[0],xvalid.shape[0],dic[tt]) 
   model=model_create(number) 
   model=model_fit(model,xtrain,xvalid,ytrain,yvalid)
   yt,pr,acc=model_test(model,xtrain,xtest,ytrain,ytest)
  
  
   arg=pr.argmax(axis=1)
   diction={0:"walking",1:"running"}
   l=[0,0]
   for i in range(len(arg)):
    l[arg[i]]+=1
   max=0  
   for i in range(1,len(l)):
     if l[i]>l[max]:
       max=i
   print(diction[max],"with ",acc*100,"% accuracy")

   tw=283
   tr=282
   vw=269
   vr=268
   te=268
   tt="r"
   dic={"w":0,"r":1}
   number=100


   xtrain,xtest,xvalid=im_data(tw,tr,te,vw,vr,number)
   ytrain,ytest,yvalid=label_create(xtrain.shape[0],xtest.shape[0],xvalid.shape[0],dic[tt]) 
   yt,pr,acc=model_test(model,xtrain,xtest,ytrain,ytest)
   arg=pr.argmax(axis=1)
   
   
   diction={0:"walking",1:"running"}
   l=[0,0]
   for i in range(len(arg)):
      l[arg[i]]+=1
   max=0  
   for i in range(1,len(l)):
     if l[i]>l[max]:
         max=i
   print(diction[max],"with ",acc*100,"% accuracy")
   return model


    
   

Running Driver Function

In [16]:
model=allwork()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, 256)                  269312    
_________________________________________________________________
dense (Dense)                (1, 64)                   16448     
_________________________________________________________________
dense_1 (Dense)              (1, 2)                    130       
Total params: 285,890
Trainable params: 285,890
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
400/400 - 27s - loss: 0.6398 - accuracy: 0.5000 - val_loss: 0.6164 - val_accuracy: 0.5100
Epoch 2/10
400/400 - 25s - loss: 0.5276 - accuracy: 0.5875 - val_loss: 195844.2656 - val_accuracy: 0.6800
Epoch 3/10
400/400 - 25s - loss: 1319824785408.0000 - accuracy: 0.8200 - val_loss: 0.5154 - val_accuracy: 0.7900
Epoch 4/10
400/400 - 25s - loss: 0.4688 - accuracy: 0.8